In [1]:
from unstructured.partition.pdf import partition_pdf
from langchain.docstore.document import Document
import tempfile

c:\Software_Tools\Anaconda\envs\ADRAG\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def pdf_processor1(bytes):
        with tempfile.NamedTemporaryFile(delete=False, suffix=".pdf") as tmp:
            tmp.write(bytes)
            tmp.flush()                  
            tmp_path = tmp.name
            raw_pdf_elements=partition_pdf(
            filename=tmp_path,         # mandatory
            strategy="hi_res",                                 # mandatory to use ``hi_res`` strategy
            extract_images_in_pdf=True,                       # mandatory to set as ``True``
            extract_image_block_types=["Image", "Table"],          # optional
            extract_image_block_to_payload=False)              # optiona
                        
        return raw_pdf_elements 

In [3]:
file_path = r"startupai-financial-report.pdf"
with open(file_path, "rb") as f:
    bytes = f.read()

In [3]:
file_path = r"startupai-financial-report.pdf"
output_dir = r"all_images"


In [4]:
raw_pdf_elements = pdf_processor1(bytes)

In [5]:
raw_pdf_elements

In [6]:
def extract_text_elements(raw_pdf_elements):
    Header, Footer, Title , NarrativeText, Text , ListItem , Img , Tables =  [], [], [],  [], [], [], [], []
    for element in raw_pdf_elements:
        if "unstructured.documents.elements.Header" in str(type(element)):
             Header.append(str(element))
        elif "unstructured.documents.elements.Footer" in str(type(element)):
             Footer.append(str(element))
        elif "unstructured.documents.elements.Title" in str(type(element)):
             Title.append(str(element))
        elif "unstructured.documents.elements.NarrativeText" in str(type(element)):
             NarrativeText.append(str(element))
        elif "unstructured.documents.elements.Text" in str(type(element)):
             Text.append(str(element))
        elif "unstructured.documents.elements.ListItem" in str(type(element)):
             ListItem.append(str(element))
        elif "unstructured.documents.elements.Image" in str(type(element)):
             Img.append(str(element))
        elif "unstructured.documents.elements.Table" in str(type(element)):
             Tables.append(str(element))
        elif 'unstructured.documents.elements.CompositeElement' in str(type(element)):
             Text.append(str(element))
    return Header, Footer, Title , NarrativeText, Text , ListItem , Img , Tables

In [7]:
Header, Footer, Title , NarrativeText, Text , ListItem , Img , Tables = extract_text_elements(raw_pdf_elements)

In [9]:
Text


['2020 2021 2022 2023 2024']

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
api_endpoint = os.getenv("ASTRA_DB_API_ENDPOINT")
app_token = os.getenv("ASTRA_DB_APPLICATION_TOKEN")

In [3]:
from langchain_astradb import AstraDBVectorStore

c:\Software_Tools\Anaconda\envs\openAI\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
from langchain_astradb import AstraDBVectorStore
from langchain.schema import Document
import os
import torch
import clip
from PIL import Image
from langchain.embeddings.base import Embeddings
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="clip")


device = "cuda" if torch.cuda.is_available() else "cpu"
# Load and save weights locally
model, preprocess = clip.load(
    "ViT-B/32",
    download_root="C:/models/clip_weights"  # <-- this folder will store weights
)

class CLIPEmbeddings(Embeddings):
    def __init__(self, model, preprocess, device):
        self.model = model
        self.preprocess = preprocess
        self.device = device

    def embed_query(self, text: str = None, **kwargs):
        if text is None:
            raise ValueError("No text provided to embed_query")

        if os.path.exists(text) and text.lower().endswith(('.png', '.jpg', '.jpeg')):
            image = Image.open(text).convert("RGB")
            return self.embed_image(image)
        else:
            return self.embed_text(text)

    def embed_documents(self, texts_or_paths):
        """
        Handle list of text or image paths
        """
        embeddings = []
        for item in texts_or_paths:
            embeddings.append(self.embed_query(item))
        return embeddings

    def embed_text(self, text: str):
        """Embed text only"""
        text_features = clip.tokenize([text], truncate=True).to(self.device)
        with torch.no_grad():
            emb = self.model.encode_text(text_features).cpu().numpy()[0]
        return emb.tolist()

    def embed_image(self, image: Image.Image):
        """Embed a PIL image"""
        image_tensor = self.preprocess(image).unsqueeze(0).to(self.device)
        with torch.no_grad():
            emb = self.model.encode_image(image_tensor).cpu().numpy()[0]
        return emb.tolist()


# Initialize embeddings
clip_embeddings = CLIPEmbeddings(model, preprocess, device)

In [5]:
vector_store = AstraDBVectorStore(
    collection_name = "RAG",
    embedding = clip_embeddings,
    api_endpoint = api_endpoint,
    token = app_token,
    pre_delete_collection=False,
    metadata_indexing_include=["source", "type", "doc_id"]
)

In [10]:
astra_index = {'Collection_name' : "Chetan"}

In [11]:
user_collections = {}

In [12]:
user_collections["user_id"] = astra_index

In [15]:
user_collections

{}

In [14]:
del user_collections['user_id']

In [ ]:
vector_store.collection_name

AsyncAstraDBCollection[astra_db="AsyncAstraDB[endpoint="https://59ec24d5-66e0-4139-a964-ac6b4c4c6ca0-us-east-2.apps.astra.datastax.com", keyspace="default_keyspace"]", collection_name="RAG"]

In [19]:
from astrapy import DataAPIClient

In [20]:
database = DataAPIClient(app_token).get_database(api_endpoint)

In [22]:
database.list_collection_names()

['Multimodal_RAG']

In [ ]:
database.drop_collection()